# 🧠 DBSI-Optimized: Tutorial Notebook

**Diffusion Basis Spectrum Imaging - High Performance Implementation**

---

## 1. Data Loading



In [ ]:
import dbsi_optimized
import numpy as np
import matplotlib.pyplot as plt
from dbsi_optimized.preprocessing import load_dwi_data

dwi_file='DWI_preprocessed_N4.nii.gz'
bvec_file='DWI_corrected.bvec'
bval_file='DWI_corrected.bval'
mask_file='DWI_preprocessed_brain_mask.nii.gz'

dwi, bvals, bvecs, mask, affine = load_dwi_data(dwi_file, bval_file, bvec_file, mask_file)

---
## 2. SNR estimation + model parameters calibration + model fitting

Based on the estimated SNR, calibration of Number of isotropic bases and Regularization is performed:

In [ ]:
from dbsi_optimized import DBSI_FastModel, estimate_snr_robust
from dbsi_optimized.calibration import run_hyperparameter_optimization

print("SNR Estimation...\n")

snr_result = estimate_snr_robust(dwi, bvals, mask)

print(f" SNR final: {snr_result['snr']:.2f}")
print(f" Method: {snr_result['method_used']}")


opt_params = run_hyperparameter_optimization(
    bvals=bvals,
    bvecs=bvecs,
    snr=snr_result['snr'],
    seed=42, #fixed seed for reproducibility
    plot=True
)

from dbsi_optimized import plot_design_matrix

print("\n Desing Matrix Visual Inspection:")

plot_design_matrix(
    bvals=bvals,
    bvecs=bvecs,
    n_iso_bases=opt_params['best_n_bases']
)


---
## 3. Model fitting + DBSI maps saving

Fit the model and save the estimated microparameters maps:

In [ ]:
print(" DBSI model fitting...")

# Inizializza il modello con i parametri calibrati
model = DBSI_FastModel(
    n_iso_bases=opt_params['best_n_bases'], 
    reg_lambda=opt_params['best_lambda'],   
    verbose=True
)

results = model.fit(dwi, bvals, bvecs, mask)

# Salva i risultati
output_dir = "dbsi_results"
results.save(output_dir, affine=affine)

print(f"\n Results saved in '{output_dir}/'")